# Univariate AUPRC Statistics

In [13]:
from pathlib import Path

import numpy as np
import pandas as pd
from sklearn.metrics import average_precision_score
from scipy import stats
from statsmodels.stats.multitest import multipletests

In [ ]:
univariate_auprc = pd.read_csv(
    "../results/neonatal/metabolite_scores/neonatal_metabolite_aupr.csv"
)
nb_save_prefix = "0.0.3-univariate_auprc_stats"

In [10]:
auprc_tall = univariate_auprc.melt(
    id_vars=["metabolite"], var_name="outcome", value_name="auprc"
).sort_values(['outcome', 'auprc'], ascending=[True, False])
auprc_tall

,metabolite,outcome,auprc
4,C04,bpd_any,0.564806
5,C05,bpd_any,0.552939
41,RL_A,bpd_any,0.534727
39,RF_C,bpd_any,0.532321
43,TYR,bpd_any,0.531440
...,...,...,...
160,C181,rop_any,0.523337
152,C12,rop_any,0.520358
180,RO_C,rop_any,0.519239
154,C14,rop_any,0.512749


In [14]:
def calculate_auprc_empirical_pvals(metabolite_data, outcome_data, n_permutations=10000):
    """
    Calculate empirical P values for AUPRC using permutation of outcome labels
    
    Parameters:
    metabolite_data: DataFrame with metabolite features
    outcome_data: DataFrame with outcome labels
    n_permutations: Number of permutations
    
    Returns:
    DataFrame with empirical P values
    """
    
    results = []
    
    # Get outcome columns (assume they end with '_any')
    outcome_cols = [col for col in outcome_data.columns if col.endswith('_any')]
    
    for outcome in outcome_cols:
        outcome_vector = outcome_data[outcome].values
        
        # Skip if no positive cases
        if outcome_vector.sum() == 0:
            continue
        
        for feature in metabolite_data.columns:
            if feature in outcome_cols:  # Skip outcome columns
                continue
                
            feature_vector = metabolite_data[feature].values
            
            # Handle missing values
            valid_mask = ~(pd.isna(feature_vector) | pd.isna(outcome_vector))
            if valid_mask.sum() == 0:
                continue
                
            feature_clean = feature_vector[valid_mask]
            outcome_clean = outcome_vector[valid_mask]
            
            # Calculate observed AUPRC
            obs_auprc = average_precision_score(outcome_clean, feature_clean)
            
            # Generate null distribution
            null_auprcs = []
            for _ in range(n_permutations):
                # Permute outcome labels
                permuted_outcome = np.random.permutation(outcome_clean)
                null_auprc = average_precision_score(permuted_outcome, feature_clean)
                null_auprcs.append(null_auprc)
            
            null_auprcs = np.array(null_auprcs)
            
            # Calculate empirical P value (one-tailed: testing if observed > null)
            empirical_pval = np.sum(null_auprcs >= obs_auprc) / n_permutations
            
            # Adjust P=0 to be 1/n_permutations
            if empirical_pval == 0:
                empirical_pval = 1 / n_permutations
            
            results.append({
                'metabolite': feature,
                'outcome': outcome,
                'auprc': obs_auprc,
                'empirical_pval': empirical_pval,
                'null_mean': null_auprcs.mean(),
                'null_std': null_auprcs.std()
            })
    
    return pd.DataFrame(results)

In [16]:
# Load the metabolite data (you'll need to adjust paths as needed)
metabolite_data = pd.read_csv(
    "../data/processed/neonatal_conditions.csv"
).set_index("row_id")

meta = pd.read_csv(
    "../data/processed/metadata.csv", low_memory=False
).set_index("row_id")

# Filter for preterm cohort
cohort_preterm_ga = [
    "22_23",
    "24_25",
    "26_27",
    "28_29",
]
preterm_ids = meta.query("gacat in @cohort_preterm_ga").index
preterm_cohort_metab = metabolite_data.loc[preterm_ids]

# Define outcome columns
outcome_cols = ["bpd_any", "rop_any", "nec_any", "ivh_any"]

In [ ]:
# Calculate empirical P values
# Which is overkill for known null AUPRC to be outcome prevalence
# But we can still derive a P value from it
# Runtime: 10m27s with 1000 permutations
auprc_pvals = calculate_auprc_empirical_pvals(
    preterm_cohort_metab.drop(columns=outcome_cols),
    preterm_cohort_metab[outcome_cols],
    n_permutations=1000
)

In [18]:
# Apply FDR correction
auprc_pvals["empirical_pval_fdr"] = multipletests(
    auprc_pvals["empirical_pval"], method="fdr_bh"
)[1]

# Merge with your existing auprc_tall dataframe
auprc_with_stats = pd.merge(
    auprc_tall,
    auprc_pvals[['metabolite', 'outcome', 'empirical_pval', 'empirical_pval_fdr', 'null_mean', 'null_std']],
    on=['metabolite', 'outcome'],
    how='left'
)

# Sort by outcome and P value
auprc_with_stats = auprc_with_stats.sort_values(
    ['outcome', 'empirical_pval_fdr', 'auprc'],
    ascending=[True, True, False]
)

In [19]:
auprc_with_stats

,metabolite,outcome,auprc,empirical_pval,empirical_pval_fdr,null_mean,null_std
0,C04,bpd_any,0.564806,0.001,0.004,0.309879,0.004057
1,C05,bpd_any,0.552939,0.001,0.004,0.309895,0.003988
2,RL_A,bpd_any,0.534727,0.001,0.004,0.310092,0.003783
3,RF_C,bpd_any,0.532321,0.001,0.004,0.309910,0.003804
4,TYR,bpd_any,0.531440,0.001,0.004,0.310122,0.003969
...,...,...,...,...,...,...,...
179,C181,rop_any,0.523337,0.999,1.000,0.422937,0.004320
180,C12,rop_any,0.520358,1.000,1.000,0.422739,0.003852
181,RO_C,rop_any,0.519239,1.000,1.000,0.422522,0.004281
182,C14,rop_any,0.512749,1.000,1.000,0.422709,0.004201


In [21]:
# Save results
nb_save_prefix = "0.1-univariate_auprc"
auprc_with_stats.to_csv(
    f"./intermediate_output/{nb_save_prefix}_empirical_pvals.csv",
    index=False
)

In [22]:
signif_auprc = auprc_with_stats.query(
    "empirical_pval_fdr < 0.05"
)
signif_auprc

,metabolite,outcome,auprc,empirical_pval,empirical_pval_fdr,null_mean,null_std
0,C04,bpd_any,0.564806,0.001,0.004000,0.309879,0.004057
1,C05,bpd_any,0.552939,0.001,0.004000,0.309895,0.003988
2,RL_A,bpd_any,0.534727,0.001,0.004000,0.310092,0.003783
3,RF_C,bpd_any,0.532321,0.001,0.004000,0.309910,0.003804
4,TYR,bpd_any,0.531440,0.001,0.004000,0.310122,0.003969
5,C05DC,bpd_any,0.527208,0.001,0.004000,0.309924,0.003714
6,CIT,bpd_any,0.526840,0.001,0.004000,0.309767,0.003728
7,R8_10,bpd_any,0.520020,0.001,0.004000,0.310552,0.004019
8,C051,bpd_any,0.513381,0.001,0.004000,0.309979,0.003463
10,C03DC,bpd_any,0.506675,0.001,0.004000,0.309879,0.003577


In [31]:
metab_labels = pd.read_csv(
    "../config/metabolite_labels.csv"
)
metab_labels.rename(
    columns={"raw_feature_name": "metabolite"},
    inplace=True
)

In [32]:
metab_labels.head()

,metabolite,feature_label,description,category,acyl_chain_length
0,ALA,ALA,Alanine,amino acid,NaN
1,ARG,ARG,Arginine,amino acid,NaN
2,C02,C-2,C-2,short-chain acylcarnitine,2.0
3,C03,C-3,C-3,short-chain acylcarnitine,2.0
4,C03DC,C-3DC,C-3DC,short-chain acylcarnitine,3.0


In [35]:
signif_auprc_labeled = pd.merge(
    signif_auprc,
    metab_labels,
    on="metabolite"
)
signif_counts = signif_auprc_labeled.groupby(
    ["outcome", "category"]).size()
signif_counts.reset_index(name="n_signif_metab")

,outcome,category,n_signif_metab
0,bpd_any,3-hydroxy long-chain acylcarnitine,2
1,bpd_any,amino acid,4
2,bpd_any,free carnitine,1
3,bpd_any,long-chain acylcarnitine,3
4,bpd_any,short-chain acylcarnitine,6
5,ivh_any,3-hydroxy long-chain acylcarnitine,3
6,ivh_any,amino acid,9
7,ivh_any,free carnitine,1
8,ivh_any,long-chain acylcarnitine,1
9,ivh_any,short-chain acylcarnitine,4


In [36]:
def test_category_enrichment(signif_df, all_df, metab_labels):
    """
    Test for enrichment of metabolite categories in significant results
    
    Parameters:
    signif_df: DataFrame with significant metabolites
    all_df: DataFrame with all tested metabolites
    metab_labels: DataFrame with metabolite category labels
    
    Returns:
    DataFrame with enrichment test results for each category
    """
    
    # Merge category labels
    signif_labeled = pd.merge(signif_df, metab_labels, on='metabolite', how='left')
    all_labeled = pd.merge(all_df, metab_labels, on='metabolite', how='left')
    
    results = []
    
    for outcome in signif_labeled['outcome'].unique():
        # Get metabolites for this outcome
        signif_outcome = signif_labeled[signif_labeled['outcome'] == outcome]
        all_outcome = all_labeled[all_labeled['outcome'] == outcome]
        
        # Get all categories
        categories = all_labeled['category'].dropna().unique()
        
        for category in categories:
            # Build 2x2 contingency table
            # Rows: Significant / Not Significant
            # Cols: In Category / Not In Category
            
            n_signif_in_cat = (signif_outcome['category'] == category).sum()
            n_signif_not_in_cat = len(signif_outcome) - n_signif_in_cat
            
            n_not_signif_in_cat = ((all_outcome['category'] == category) & 
                                   (~all_outcome['metabolite'].isin(signif_outcome['metabolite']))).sum()
            n_not_signif_not_in_cat = (len(all_outcome) - len(signif_outcome) - 
                                       n_not_signif_in_cat)
            
            # Contingency table
            table = np.array([
                [n_signif_in_cat, n_signif_not_in_cat],
                [n_not_signif_in_cat, n_not_signif_not_in_cat]
            ])
            
            # Fisher's exact test (better for small counts)
            odds_ratio, p_value = stats.fisher_exact(table, alternative='greater')
            
            # Calculate proportions
            prop_signif = n_signif_in_cat / len(signif_outcome) if len(signif_outcome) > 0 else 0
            prop_all = (n_signif_in_cat + n_not_signif_in_cat) / len(all_outcome) if len(all_outcome) > 0 else 0
            
            # Fold enrichment
            fold_enrichment = prop_signif / prop_all if prop_all > 0 else np.inf
            
            results.append({
                'outcome': outcome,
                'category': category,
                'n_signif_in_category': n_signif_in_cat,
                'n_total_in_category': n_signif_in_cat + n_not_signif_in_cat,
                'n_signif_total': len(signif_outcome),
                'n_total': len(all_outcome),
                'prop_signif': prop_signif,
                'prop_background': prop_all,
                'fold_enrichment': fold_enrichment,
                'odds_ratio': odds_ratio,
                'p_value': p_value
            })
    
    return pd.DataFrame(results)

# Run enrichment test
enrichment_results = test_category_enrichment(
    signif_auprc,
    auprc_with_stats,
    metab_labels
)

# Apply FDR correction
enrichment_results['p_value_fdr'] = multipletests(
    enrichment_results['p_value'], 
    method='fdr_bh'
)[1]

# Sort by significance
enrichment_results = enrichment_results.sort_values(
    ['outcome', 'p_value']
)

enrichment_results

,outcome,category,n_signif_in_category,n_total_in_category,n_signif_total,n_total,prop_signif,prop_background,fold_enrichment,odds_ratio,p_value,p_value_fdr
0,bpd_any,short-chain acylcarnitine,6,10,16,46,0.375000,0.217391,1.725000,3.900000,0.066691,0.444606
3,bpd_any,3-hydroxy long-chain acylcarnitine,2,4,16,46,0.125000,0.086957,1.437500,2.000000,0.433986,1.000000
2,bpd_any,free carnitine,1,2,16,46,0.062500,0.043478,1.437500,1.933333,0.579710,1.000000
4,bpd_any,long-chain acylcarnitine,3,13,16,46,0.187500,0.282609,0.663462,0.461538,0.920809,1.000000
1,bpd_any,amino acid,4,17,16,46,0.250000,0.369565,0.676471,0.435897,0.941420,1.000000
6,ivh_any,amino acid,9,17,18,46,0.500000,0.369565,1.352941,2.500000,0.123980,0.619899
8,ivh_any,3-hydroxy long-chain acylcarnitine,3,4,18,46,0.166667,0.086957,1.916667,5.400000,0.158765,0.635058
5,ivh_any,short-chain acylcarnitine,4,10,18,46,0.222222,0.217391,1.022222,1.047619,0.612603,1.000000
7,ivh_any,free carnitine,1,2,18,46,0.055556,0.043478,1.277778,1.588235,0.634783,1.000000
9,ivh_any,long-chain acylcarnitine,1,13,18,46,0.055556,0.282609,0.196581,0.078431,0.999632,1.000000


In [38]:
# Filter for significant enrichments
signif_enrichment = enrichment_results[
    enrichment_results['p_value_fdr'] < 0.05
].copy()

# Format for presentation
signif_enrichment['enrichment_str'] = signif_enrichment.apply(
    lambda row: f"{row['fold_enrichment']:.2f}x ({row['n_signif_in_category']}/{row['n_total_in_category']})",
    axis=1
)

# Create summary table
summary = signif_enrichment[[
    'outcome', 'category', 'n_signif_in_category', 'n_total_in_category',
    'fold_enrichment', 'p_value', 'p_value_fdr'
]].round(4)
summary

,outcome,category,n_signif_in_category,n_total_in_category,fold_enrichment,p_value,p_value_fdr
15,rop_any,short-chain acylcarnitine,8,10,2.3,0.0015,0.0292
